# YYC traffic incident prediction

This noteboook is a description and implementaion of using machine learning to predict traffic indicents in the city of Calgary using the publicaly available traffic incident data

Given the location and the average temprature for the day this model will predict the probability of incident

### Lets load the incident data from the two different files into a single file
Sice we only have data from 2016 there won't be a ton of data

In [1]:
import pandas as pd
import numpy as np


In [2]:
def format_description(description):
    if "multi" in description.lower():
        return "multi"
    elif "pedestrian" in description.lower():
        return "pedestrian"
    elif "2" in description.lower():
        return "double"
    elif "single" in description.lower():
        return "single"

In [79]:
f2017 = pd.read_csv('data/2017.csv')
f2016 = pd.read_csv('data/2016.csv')
data = pd.concat([f2017, f2016])

print ('Number of rows 2016: {}'.format(len(f2016)))
print ('Number of rows 2017: {}'.format(len(f2017)))
print ('Number of rows total: {}'.format(data.shape))

# formats lat and long to have 4 decimals only
data['Longitude'] = data['Longitude'].map('{:.3f}'.format).astype(float)
data['Latitude'] = data['Latitude'].map('{:.3f}'.format).astype(float)

# removes time from date since we are not going to use the time
data['START_DT'] = data['START_DT'].str.split(' ').str[0]

# formats incident type and removes rows that do not match any type
data['incident'] = data['DESCRIPTION'].map(format_description)
data = data[data.incident.notnull()]

# creates maps from lat,long to addr and viceversa
gps_to_addr = {(lat,lon): addr for lon, lat, addr in zip(data['Longitude'],data['Latitude'], data['INCIDENT INFO'])}
addr_to_gps = {gps_to_addr[k]:k  for k in gps_to_addr.keys()}

# dropping descrption, modified, quadrant, location and count
data = data.drop(data.columns[[0,1,2,3,4,7,8]], axis=1)
data[:5]

print('Final array dims: {}'.format(data.shape))



Number of rows 2016: 317
Number of rows 2017: 1218
Number of rows total: (1535, 9)
Final array dims: (1111, 3)


### Creating the data for our model ( X and Y)

In [83]:
def load_data(data):
    y = pd.get_dummies(data['incident']) # one hot encoding for labels
    
    # scaling features
    quant_features = ['Longitude', 'Latitude']
    # Store scalings in a dictionary so we can convert back later
    scaled_features = {}
    for each in quant_features:
        mean, std = data[each].mean(), data[each].std()
        scaled_features[each] = [mean, std]
        data.loc[:, each] = (data[each] - mean)/std
    
    x = np.array(data)
    y = np.array(y)
    row = round(0.9 * x.shape[0])

    x_train = x[:int(row), :-1]
    y_train = y[:int(row)]

    x_test = x[int(row):, :-1]
    y_test = y[int(row):]
    
    print('x_train', x_train.shape)
    print('y_train', y_train.shape)
    print('x_test', x_test.shape)
    print('y_test', y_test.shape)
    
    return [x_train, y_train, x_test, y_test]

# call to create X and Y
x_train, y_train, x_test, y_test = load_data(data)

x_train (1000, 2)
y_train (1000, 4)
x_test (111, 2)
y_test (111, 4)


In [84]:
print(x_train[1], y_train[1])

[0.6770423769714381 0.19584726816943368] [0 0 1 0]


### Building model structure

In [113]:
from keras.models import Sequential
from keras.layers import Dense, Activation

def build_model(dims):
    model = Sequential()
    model.add(Dense(256, input_dim=dims[0], activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim=dims[1], activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['acc'])
    return model

In [114]:
print(x_train.shape[1])
model = build_model([x_train.shape[1],y_train.shape[1]])
model.fit(
    x_train,
    y_train,
    batch_size=12,
    epochs=300,
    validation_split=0.3,
    verbose=1)

2


/Users/daniel/Documents/personal/udacity/projects/venv/lib/python3.5/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4, activation="sigmoid")`


Train on 700 samples, validate on 300 samples
Epoch 1/300
700/700 [==============================] - 0s - loss: 0.6143 - acc: 0.7246 - val_loss: 0.5489 - val_acc: 0.7300
Epoch 2/300
700/700 [==============================] - 0s - loss: 0.4955 - acc: 0.7636 - val_loss: 0.5324 - val_acc: 0.7200
Epoch 3/300
700/700 [==============================] - 0s - loss: 0.4842 - acc: 0.7657 - val_loss: 0.5131 - val_acc: 0.7200
Epoch 4/300
700/700 [==============================] - 0s - loss: 0.4787 - acc: 0.7693 - val_loss: 0.5122 - val_acc: 0.7242

In [115]:
import math
trainScore = model.evaluate(x_train, y_train, verbose=0)
print(trainScore)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(x_test, y_test, verbose=0)
print(testScore)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

[0.57671810603141782, 0.77100000000000002]
Train Score: 0.58 MSE (0.76 RMSE)
[0.4491937724319664, 0.79504504719296021]
Test Score: 0.45 MSE (0.67 RMSE)


In [122]:
p = model.predict(x_test[:5])
print(p)
print(y_test[:5])
print(np.argmax(p[3]), np.argmax(y_test[3]))

[[ 0.45269233  0.18471643  0.0530221   0.28821072]
 [ 0.60449517  0.15033983  0.0803412   0.13770461]
 [ 0.34699488  0.10877071  0.02987239  0.42157647]
 [ 0.55727142  0.23666367  0.0162794   0.1702114 ]
 [ 0.57544559  0.18778726  0.01939066  0.21215332]]
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]]
0 1


In [145]:


data.groupby(['Longitude','Latitude']).count()

INCIDENT INFO  DESCRIPTION  START_DT  QUADRANT  incident
Longitude Latitude                                                          
-113.919  51.068                1            1         1         1         0
-113.920  51.068                1            1         1         1         0
          51.073                1            1         1         1         1
-113.922  51.137                1            1         1         1         0
          51.144                1            1         1         1         0
-113.929  50.949                2            2         2         2         0
          50.980                2            2         2         2         0
          51.038                2            2         2         2         1
          51.154                2            2         2         2         0
-113.930  51.015                3            3         3         3         2
          51.067               11           11        11        11         2
-113.933  50.923                1            1         1         1         1
          51.111                1            1         1         1         0
-113.935  50.980                2            2         2         2         2
          51.046                2            2         2         2         1
          51.052                1            1         1         1         1
          51.060                1            1         1         1         1
          51.108                2            2         2         2         2
          51.111                2            2         2         2         1
          51.154                2            2         2         2         1
-113.936  51.052                5            5         5         5         3
          51.082                1            1         1         1         0
          51.092                2            2         2         2         2
-113.937  51.096                3            3         3         3         1
          51.125                2            2         2         2         2
-113.938  50.914                1            1         1         1         1
          51.067                2            2         2         2         2
-113.941  51.082                2            2         2         2         1
-113.942  50.885                1            1         1         1         0
          50.906                1            1         1         1         0
...                           ...          ...       ...       ...       ...
-114.189  51.081                1            1         1         1         0
-114.195  51.127                2            2         2         2         2
-114.196  51.014                2            2         2         2         0
-114.198  51.124                2            2         2         2         2
-114.199  51.060                1            1         1         1         1
          51.067                1            1         1         1         1
-114.203  51.018                2            2         2         2         2
          51.120                2            2         2         2         1
-114.204  51.118                1            1         1         1         1
-114.205  51.118                1            1         1         1         1
-114.207  51.116                2            2         2         2         0
-114.209  51.084                1            1         1         1         0
-114.210  51.147                3            3         3         3         2
-114.211  51.038                1            1         1         1         0
          51.052                1            1         1         1         0
          51.065                1            1         1         1         0
          51.067                2            2         2         2         2
-114.213  51.125                1            1         1         1         0
-114.214  51.150                1            1         1         1         1
-114.216  51.085